In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
!python --version

In [ ]:

# Define data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1.0/255.0,  # Rescale pixel values to [0, 1]
    validation_split=0.2  # Split data into training and validation sets
)

In [ ]:

# Load and preprocess the dataset
train_generator = datagen.flow_from_directory(
    r"split_chest_xray/train2",
    target_size=(224, 224),  # Xception requires input size of (224, 224)
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

In [ ]:

validation_generator = datagen.flow_from_directory(
    r"split_chest_xray/val",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
# Load and preprocess the dataset for testing
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_directory(
    r"split_chest_xray/test",  # Replace with the path to your testing dataset directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
base_model = Xception(weights='imagenet', include_top=False)

In [ ]:
labels = train_generator.class_indices
with open('class_labels.txt', 'w') as file:
    file.write("[")
    for l in labels:
        file.write(f"{l}, ")
    file.write("]")

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(train_generator.num_classes, activation='softmax')(x)

In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=outputs)


In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Train the model
EPOCHS = 3

# Define the early stopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
early_stopping = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    # validation_data=validation_generator,
    callbacks=[early_stopping]
)


In [ ]:
# Plot training and validation accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
# plt.title('Training and Validation Accuracy')
plt.title('Training')
plt.show()

In [ ]:
# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


In [ ]:
# Save the model for future use
model.save(r"xception_train2.h5")

In [ ]:
tf.__version__